<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports-&amp;-Declarations" data-toc-modified-id="Imports-&amp;-Declarations-1">Imports &amp; Declarations</a></span></li><li><span><a href="#Define-the-geographic-area-of-interest" data-toc-modified-id="Define-the-geographic-area-of-interest-2">Define the geographic area of interest</a></span></li><li><span><a href="#Retireve-Foursquare-venues" data-toc-modified-id="Retireve-Foursquare-venues-3">Retireve Foursquare venues</a></span></li><li><span><a href="#Export-data" data-toc-modified-id="Export-data-4">Export data</a></span></li></ul></div>

# Getting to know Rexburg
**Where to live, where to eat, what to do**

_Coursera Applied Data Science Capstone Project, January 2020_

## Imports & Declarations

In [1]:
import numpy as np
import pandas as pd

import folium
from folium.features import DivIcon

import capstone_functions as cf

## Define the geographic area of interest

Rexburg is a small town in the state of Idaho in the United States' northern intermountain west.

![Rexburg, ID](assets/rex_regional_small.png "Rexburg, ID")
  
  
  Since our objective is to help a prospective college student who'll be moving to Rexburg get familiar with the town's resdiential, food, and recreation options, we'll arbitrarily choose a $4km \times 4km$ square centered around the college campus.

The coordinates of the four corners of this square are:


| Lat/Long | West                     | East                     |
|-------|--------------------------|--------------------------|
| <b>North | 43.840199<br>-111.807453 | 43.840199<br>-111.757579 |
| <b>South | 43.804226<br>-111.807453 | 43.804226<br>-111.757579 |

_(Please note that the remaining map visualizations in this project will not render when previewed in github. You'll have to run the code to see them.)_

In [2]:
cf.draw_rexburg_map()

## Retireve Foursquare venues

We'll be using the [Foursquare API](https://developer.foursquare.com/places) to retrieve information about places in Rexburg. The `explore` endpoint will return venues in all categories centered on a set of coordinates within a radius we can set.

Since each call is is limited to 100 results, we want to search an area small enough that we'll capture as many venues as possible. The "sandbox" version of the API allows 950 "regular" API calls (of which `explore` is one) per day.

To balance these two priorities, we'll split the area of interest into a $20\times20$ grid ($200m\times200m)$, and call the API from the center of each grid section. That should be a small enough area that we don't hit the limit of 100 venues per API request and only use up 400 of our 950 daily calls.

In [3]:
grid_size = 20
cf.draw_rexburg_map(grid_size=grid_size)

We'll need to specify a radius large enough to inscribe each grid section in a circle. The searches will overlap, as shown below, so we'll end up with a substantial number of duplicates, as well as venues outside the border, which we'll eliminate before saving our data.

In [7]:
cf.draw_rexburg_map(zoom_start=14, grid_size=grid_size, circles=True, grid=False)

Let's retrieve the venues from Foursquare.  (The functions being used are available in the `capstone_functions.py` file located in the main project directory.

In [8]:
rexburg_venues = cf.get_nearby_venues(grid_size)
print(f'\n\n{rexburg_venues.shape[0]} venues retrieved.')

Getting grid sections: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 2

And now we'll drop duplicate venues, based on name and coordinates.

In [9]:
rexburg_venues.drop_duplicates(['venue', 'v_lat', 'v_long'], inplace=True)
print(f'{rexburg_venues.shape[0]} remain after dropping duplicates.')

900 remain after dropping duplicates.


Now we'll eliminate any venues outside the borders. Since we only have to do this once, we'll  use a function we've already created, even though the method is slightly inefficient. 

First, we'll determine the grid number for each of the venues in our dataframe, based on its coordinates.

In [11]:
for row in rexburg_venues.itertuples():
    rexburg_venues.loc[
        row.Index, 'grid_section'] = cf.venue_grid_section(row.v_lat, 
                                                           row.v_long, 
                                                           grid_size)

Any venue that's not in a grid section will be assigned `grid_section = NaN`.

In [12]:
rexburg_venues.grid_section.isnull().sum()

107

 Then we'll drop the rows with null grid sections.

In [13]:
rexburg_venues.dropna(inplace=True)
rexburg_venues.isnull().sum().sum()

0

In [14]:
print(f'This leaves us with {rexburg_venues.shape[0]} venues in the final dataset.')

This leaves us with 793 venues in the final dataset.


In [15]:
map_venues = cf.draw_rexburg_map(grid=False)

for lat, long, label in zip(rexburg_venues.v_lat, 
                            rexburg_venues.v_long, 
                            rexburg_venues.venue):
    folium.CircleMarker(
        [lat, long],
        radius=3,
        popup=label,
        fill=True,
        color='black',
        fill_color='black',
        fill_opacity=0.25,
        weight=1
        ).add_to(map_venues)
    
map_venues

## Export data

Let's save those venues to a file we can use for the remaining analysis.

In [14]:
rexburg_venues.drop('grid_section').to_csv('data/rexburg_venues.csv', index=False)

----